# Setup 🏗️


In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split




In [2]:
viscos = 1/5200

# load RANS data created by rans.m (which can be downloaded)
# load DNS data
DNS_mean  = np.genfromtxt("LM_Channel_5200_mean_prof.dat",comments="%").transpose()
y_DNS     = DNS_mean[0]
yplus_DNS = DNS_mean[1]
u_DNS     = DNS_mean[2]
dudy_DNS  = np.gradient(u_DNS,y_DNS)

DNS_stress = np.genfromtxt("LM_Channel_5200_vel_fluc_prof.dat",comments="%").transpose()

uu_DNS = DNS_stress[2]
vv_DNS = DNS_stress[3]
ww_DNS = DNS_stress[4]
uv_DNS = DNS_stress[5]
uw_DNS = DNS_stress[6]
vw_DNS = DNS_stress[7]
k_DNS  = 0.5*(uu_DNS+vv_DNS+ww_DNS)

DNS_RSTE = np.genfromtxt("LM_Channel_5200_RSTE_k_prof.dat",comments="%")

eps_DNS = DNS_RSTE[:,7]/viscos # it is scaled with ustar**4/viscos

# fix wall
eps_DNS[0]=eps_DNS[1]

# load data from k-omega RANS
data  = np.loadtxt('y_u_k_om_uv_5200-RANS-code.txt').transpose()
y     = data[0]
u     = data[1]
k     = data[2]
om    = data[3]
diss1 = 0.09*k*om
ustar = (viscos*u[0]/y[0])**0.5
yplus = y*ustar/viscos

# dont train on, uu, vv, ww, uv, uw, vw 
# Maybe mixed terms are ok (just not uu,vv,ww)

#-----------------Data_manipulation--------------------

# Delete first value for all interesting data
uv_DNS    = np.delete(uv_DNS, 0)
vv_DNS    = np.delete(vv_DNS, 0)
ww_DNS    = np.delete(ww_DNS, 0)
uw_DNS    = np.delete(uw_DNS,0)
vw_DNS    = np.delete(vw_DNS,0)
k_DNS     = np.delete(k_DNS, 0)
eps_DNS   = np.delete(eps_DNS, 0)
dudy_DNS  = np.delete(dudy_DNS, 0)
yplus_DNS = np.delete(yplus_DNS,0)
uu_DNS    = np.delete(uu_DNS,0)
y_DNS     = np.delete(y_DNS,0)
u_DNS     = np.delete(u_DNS,0)

# Calculate ny_t and time-scale tau
viscous_t = k_DNS**2/eps_DNS 
tau       = viscous_t/abs(uv_DNS)

# Calculate c_1, c_2, & c_3 of the Non-linear Eddy Viscosity Model
# Array for storing c_1, c_2, & c_3
c_0 = -2*(ww_DNS/k_DNS - 2/3)/(tau**2*dudy_DNS**2)
c_2 = 2*((ww_DNS/k_DNS - 2/3) + (uu_DNS/k_DNS - 2/3))/(tau**2*dudy_DNS**2)

#TODO ML using PyTorch to estimate c_1, c_2, & c_3


In [49]:
c[2].dtype

dtype('float64')

Setting up input and output as tensors 

In [3]:
# stack several arrays horizontally to create a feature matrix
from sklearn.discriminant_analysis import StandardScaler

scaler1 = StandardScaler()
scaler2 = StandardScaler()

X = np.column_stack((y_DNS, yplus_DNS, u_DNS, dudy_DNS, k_DNS))

# transpose the target vector to make it a column vector
y = c.transpose()

# split the feature matrix and target vector into training and validation sets
# test_size=0.2 means we reserve 20% of the data for validation
# random_state=42 is a fixed seed for the random number generator, ensuring reproducibility
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler1.fit_transform(X_train)
X_val_scaled   = scaler2.fit_transform(X_val)


# convert the numpy arrays to PyTorch tensors with float32 data type
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# create PyTorch datasets and dataloaders for the training and validation sets
# a TensorDataset wraps the feature and target tensors into a single dataset
# a DataLoader loads the data in batches and shuffles the batches if shuffle=True
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



Let's set up a neural network:

In [4]:
class TestNetwork(nn.Module):

    def __init__(self):
        
        super(TestNetwork, self).__init__()

        self.input   = nn.Linear(5, 50)     # 5 inputs, 50 outputs
        self.hidden1 = nn.Linear(50, 25)    # 50 inputs, 25 outputs
        self.hidden2 = nn.Linear(25, 3)     # 25 inputs, 3 outputs

    def forward(self, x):
        x = nn.functional.relu(self.input(x))
        x = nn.functional.relu(self.hidden1(x))
        x = self.hidden2(x)
        return x
    
    # Create an instance of the network
net = TestNetwork()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Train the network
for epoch in range(100):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        # Get the inputs and labels
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0




[1,    10] loss: 0.011
[1,    20] loss: 0.011
[2,    10] loss: 0.008
[2,    20] loss: 0.007
[3,    10] loss: 0.006
[3,    20] loss: 0.005
[4,    10] loss: 0.005
[4,    20] loss: 0.004
[5,    10] loss: 0.003
[5,    20] loss: 0.003
[6,    10] loss: 0.003
[6,    20] loss: 0.003
[7,    10] loss: 0.002
[7,    20] loss: 0.002
[8,    10] loss: 0.002
[8,    20] loss: 0.002
[9,    10] loss: 0.002
[9,    20] loss: 0.002
[10,    10] loss: 0.002
[10,    20] loss: 0.002
[11,    10] loss: 0.002
[11,    20] loss: 0.002
[12,    10] loss: 0.002
[12,    20] loss: 0.001
[13,    10] loss: 0.001
[13,    20] loss: 0.002
[14,    10] loss: 0.001
[14,    20] loss: 0.001
[15,    10] loss: 0.001
[15,    20] loss: 0.001
[16,    10] loss: 0.001
[16,    20] loss: 0.001
[17,    10] loss: 0.001
[17,    20] loss: 0.001
[18,    10] loss: 0.001
[18,    20] loss: 0.001
[19,    10] loss: 0.001
[19,    20] loss: 0.001
[20,    10] loss: 0.001
[20,    20] loss: 0.001
[21,    10] loss: 0.001
[21,    20] loss: 0.001
[22,    10